# Organizing Files

## The shutil Module

### Copying Files and Folders

In [ ]:
import shutil, os
from pathlib import Path
p = Path.home()
# the original filename is used for the new, copied file's filename
# shutil.copy(p / 'Desktop/my_file5.csv', p / 'Desktop/demo/')
# give the copied file's filename a new name
# shutil.copy(p / 'Desktop/my_file5.csv', p / 'Desktop/demo/my_file6.csv')

In [ ]:
import shutil, os
from pathlib import Path
p = Path.home()
# shutil.copytree(p / 'Desktop/demo', p / 'Desktop/demo_backup')

### Moving and Renaming Files and Folders

In [ ]:
import shutil
# if destination points to a folder, the source file gets moved into destination
# assuming a folder named b already exists
# shutil.move('/Users/ljy/Desktop/demo/a/test.txt', '/Users/ljy/Desktop/demo/b/')

# move and rename
# shutil.move('/Users/ljy/Desktop/demo/b/test.txt', '/Users/ljy/Desktop/demo/a/new_test.txt')

# if a folder named c doesn't exist, new_test.txt is renamed c
# shutil.move('/Users/ljy/Desktop/demo/a/new_test.txt', '/Users/ljy/Desktop/demo/c')

# the folders that make up the destination must already exist
# shutil.move('/Users/ljy/Desktop/demo/a/new_test.txt', '/Users/does_not_exist/eggs') FileNotFoundError: [Errno 2] No such file or directory

### Permanently Deleting Files and Folders

In [ ]:
import os
from pathlib import Path
for filename in Path.home().glob('Desktop/demo/*.txt'):
    # os.unlink(filename)
    print(filename)

### Safe Deletes with the send2trash Module

In [ ]:
import send2trash
# baconFile = open('/Users/ljy/Desktop/demo/bacon.txt', 'a') # creates the file
# baconFile.write('Bacon is not a vegetable.')
# baconFile.close()
# send2trash.send2trash('/Users/ljy/Desktop/demo/bacon.txt')

## Walking a Directory Tree

In [ ]:
import os

for folderName, subfolders, filenames in os.walk('/Users/ljy/Desktop/demo/'):
    print('The current folder is ' + folderName)
    
    for subfolder in subfolders:
        print('SUBFOLDER OF ' + folderName + ': ' + subfolder)
        
    for filename in filenames:
        print('FILE INSIDE ' + folderName + ': ' + filename)
    
    print('')

## Compressing Files with the zipfile Module

### Reading ZIP Files

In [ ]:
import zipfile, os
from pathlib import Path
p = Path.home()
exampleZip = zipfile.ZipFile(p / 'Desktop/demo.zip')
print(exampleZip.namelist())

textFileInfo = exampleZip.getinfo('demo/c')
print(type(textFileInfo))
print(textFileInfo.file_size)
print(textFileInfo.compress_size)
print(f'Compressed file is {round(textFileInfo.file_size / textFileInfo.compress_size)}x smaller!')
exampleZip.close()

### Extracting from ZIP Files

In [ ]:
import zipfile, os
from pathlib import Path
p = Path.home()
exampleZip = zipfile.ZipFile(p / 'Desktop/demo.zip')
exampleZip.extractall()
exampleZip.close()

In [ ]:
exampleZip = zipfile.ZipFile(p / 'Desktop/demo.zip')
exampleZip.extract('demo/1.txt')
# exampleZip.extract('demo/1.txt', p / 'Desktop/newFolder/sub')
exampleZip.close()

### Creating and Adding to ZIP Files

In [ ]:
import zipfile
newZip = zipfile.ZipFile('new.zip', 'w')
newZip.write('demo/1.txt', compress_type=zipfile.ZIP_DEFLATED)
newZip.close()

## Project: Renaming Files with American-Style Dates to European-Style Dates

In [ ]:
# Here's what the program does:
# 1. It searches all the filenames in the current working directory for American-style dates.
# 2. When one is found, it renames the file with the month and day swapped to make it European-style.

# This means the code will need to do the following:
# 1. Create a regex that can identify the text pattern of American-style dates.
# 2. Call os.listdir() to find all the files in the working directory.
# 3. Loop over each filename, using the regex to check whether it has a date.
# 4. If it has a date, rename the file with shutil.move().

In [ ]:
# renameDates.py - Renames filenames with American MM-DD-YYYY date format to European DD-MM-YYYY

import shutil, os, re

# Create a regex that matches files with the American date format.
datePattern = re.compile(r"""^(.*?) # all text before the date
    ((0|1)?\d)-                     # one or two digits for the month
    ((0|1|2|3)?\d)-                 # one or two digits for the day
    ((19|20)\d\d)                   # four digits for the year
    (.*?)$                          # all text after the date
    """, re.VERBOSE)

# Loop over the files in the working directory.
for amerFilename in os.listdir('/Users/ljy/Desktop/demo'):
    mo = datePattern.search(amerFilename)
    
    # Skip files without a date.
    if mo == None:
        continue
        
    # Get the different parts of the filename.
    beforePart = mo.group(1)
    monthPart = mo.group(2)
    dayPart = mo.group(4)
    yearPart = mo.group(6)
    afterPart = mo.group(8)
    
    # Form the European-style filename.
    euroFilename = beforePart + dayPart + '-' + monthPart + '-' + yearPart + afterPart
    
    # Get the full, absolute file paths.
    absWorkingDir = os.path.abspath('/Users/ljy/Desktop/demo')
    amerFilename = os.path.join(absWorkingDir, amerFilename)
    euroFilename = os.path.join(absWorkingDir, euroFilename)
    
    # Rename the files.
    print(f'Renaming "{amerFilename}" to "{euroFilename}"...')
    # shutil.move(amerFilename, euroFilename) # uncomment after testing

## Project: Backing Up a Folder into a ZIP File

In [ ]:
# backupToZip.py - Copies an entire folder and its contents into a ZIP file whose filename increments.
import zipfile, os

def backupToZip(folder):
    """Back up the entire contents of "folder" into a ZIP file."""
    
    folder = os.path.abspath(folder) # make sure folder is absolute
    
    # Figure out the filename this code should use based on what files already exist.
    number = 1
    while True:
        zipFilename = os.path.basename(folder) + '_' + str(number) + '.zip'
        if not os.path.exists(zipFilename):
            break
        number += 1
    
    # Create the zip file.
    print(f'Creating {zipFilename}')
    backupZip = zipfile.ZipFile(zipFilename, 'w')
    
    # Walk the entire folder tree and compress the files in each folder.
    for foldername, subfolders, filenames in os.walk(folder):
        print(f'Adding files in {foldername}...')
        # Add the current folder to the ZIP file.
        for filename in filenames:
            newBase = os.path.basename(folder) + '_'
            if filename.startswith(newBase) and filename.endswith('.zip'):
                continue
            backupZip.write(os.path.join(foldername, filename))
    backupZip.close()
    
    print('Done.')            

backupToZip('/Users/ljy/Desktop/demo')